In [122]:
import pandas as pd

# Loading the spam data
# ham is the label for non-spam messages
spam = pd.read_excel('test.xlsx')
spam.head(10)

,Status,Decisão
0,parcialmente provido,conhecido. parcialmente provido. unanime
1,desprovido,conhecido. improvido. unânime
2,parcialmente provido,conhecido. parcialmente provido. unânime
3,desprovido,conhecer em parte. negar provimento. unânime
4,desprovido,conhecer do recurso e negar-lhe provimento. un...
5,provido,conhecido. provido. unânime.
6,desprovido,apelação da autora e recurso adesivo da ré con...
7,desprovido,conhecido. preliminar rejeitada. prejudicial r...
8,parcialmente provido,dar parcial provimento ao recurso de m.c.b.s.r...
9,desprovido,conhecer do recurso e negar-lhe provimento.


In [123]:
import spacy
nlp = spacy.blank("pt")

# Create the TextCategorizer with exclusive classes and "bow" architecture
textcat = nlp.create_pipe(
              "textcat",
              config={
                "exclusive_classes": True,
                "architecture": "bow"})

# Add the TextCategorizer to the empty model
nlp.add_pipe(textcat)

In [124]:
# Add labels to text classifier
textcat.add_label("provido")
textcat.add_label("desprovido")
textcat.add_label("parcialmente provido")
textcat.add_label("parcialmente improvido")

1

In [125]:
train_texts = spam['Decisão'].values
train_labels = [{'cats': {'provido': label == 'provido',
                          'desprovido': label == 'desprovido',
                          'parcialmente provido': label == 'parcialmente provido',
                          'parcialmente improvido': label == 'parcialmente improvido',
                         }
                } 
                for label in spam['Status']]

In [126]:
train_data = list(zip(train_texts, train_labels))
train_data[:3]

[('conhecido. parcialmente provido. unanime',
  {'cats': {'provido': False,
    'desprovido': False,
    'parcialmente provido': True,
    'parcialmente improvido': False}}),
 ('conhecido. improvido. unânime',
  {'cats': {'provido': False,
    'desprovido': True,
    'parcialmente provido': False,
    'parcialmente improvido': False}}),
 ('conhecido. parcialmente provido. unânime',
  {'cats': {'provido': False,
    'desprovido': False,
    'parcialmente provido': True,
    'parcialmente improvido': False}})]

In [127]:
from spacy.util import minibatch

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

# Create the batch generator with batch size = 8
batches = minibatch(train_data, size=8)
# Iterate through minibatches
for batch in batches:
    # Each batch is a list of (text, label) but we need to
    # send separate lists for texts and labels to update().
    # This is a quick way to split a list of tuples into lists
    texts, labels = zip(*batch)
    nlp.update(texts, labels, sgd=optimizer)

In [128]:
import random

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(10):
    random.shuffle(train_data)
    # Create the batch generator with batch size = 8
    batches = minibatch(train_data, size=8)
    # Iterate through minibatches
    for batch in batches:
        # Each batch is a list of (text, label) but we need to
        # send separate lists for texts and labels to update().
        # This is a quick way to split a list of tuples into lists
        texts, labels = zip(*batch)
        nlp.update(texts, labels, sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 0.7647732822224498}
{'textcat': 1.546765330247581}
{'textcat': 2.243905583396554}
{'textcat': 2.8919967440888286}
{'textcat': 3.7514360398054123}
{'textcat': 4.272939569316804}
{'textcat': 4.974101124331355}
{'textcat': 5.666876802220941}
{'textcat': 6.342862875666469}
{'textcat': 6.818926766049117}


In [129]:
texts = ["conhecer do recurso e negar-lhe provimento. unânime.", 
         "conhecer. dar parcial improvimento. unânime",
         "apelação parcialmente conhecida e parcialmente provida. unânime.",
         "prover-lhe provimento"]
docs = [nlp.tokenizer(text) for text in texts]   
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(docs)

print(scores)

[[0.23261581 0.427228   0.22569457 0.11446159]
 [0.27842712 0.26621705 0.23585552 0.21950027]
 [0.19843525 0.2734373  0.36695364 0.16117382]
 [0.28225625 0.2705997  0.2356218  0.2115223 ]]


In [130]:
# From the scores, find the label with the highest score/probability
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])

['desprovido', 'provido', 'parcialmente provido', 'provido']


In [135]:
df = pd.read_excel('Dados.xlsx', index=False, usecols = [7])  
my_texts = []
for i in df.values.tolist():
    my_texts.append(str(i[0]).lower())
    
print(len(my_texts))

6694


In [138]:
my_texts = my_texts[:25]
print(my_texts)
docs = [nlp.tokenizer(a) for a in my_texts]
    
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(docs)

print(scores)

['acolher a preliminar de ilegitimidade passiva dosegundo réu para julgar extinto o processo em relação a ele.dar parcial provimento ao recurso da primeira ré. unânime', 'conhecer e negar provimento, unânime', 'conhecer e dar parcial provimento, unânime', 'dar parcial provimento ao recurso do autor. darparcial provimento ao recurso dos réus. unânime', 'conhecer. negar provimento. unânime', 'conhecido. desprovido. unânime', 'conhecer do apelo da 2ª ré e negar provimento.conhecer em parte do apelo do autor e dar parcial provimento.unânime', 'conhecido. desprovido. unânime', 'conhecido. provido. unânime', 'conhecido. provido. unânime', 'conhecer. negar provimento. unânime', 'conhecer. dar parcial provimento. unânime', 'conhecer. dar parcial provimento. unânime', 'conhecer, rejeitar a(s) preliminar(es) e, no mérito,dar parcial provimento, unânime', 'conhecer do recurso da denunciada à lide e darparcial provimento. conhecer em parte do recurso do réudenunciado e negar provimento, unânime']


In [139]:
# From the scores, find the label with the highest score/probability
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])

['parcialmente provido', 'desprovido', 'parcialmente provido', 'provido', 'desprovido', 'desprovido', 'desprovido', 'desprovido', 'provido', 'provido', 'desprovido', 'desprovido', 'desprovido', 'parcialmente provido', 'desprovido']
